In [1]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
  !mkdir -p Sources
  !wget https://raw.githubusercontent.com/NVIDIA/accelerated-computing-hub/refs/heads/main/tutorials/cuda-cpp/notebooks/01.03-Extending-Algorithms/Sources/ach.h -nv -O Sources/ach.h


2026-01-28 12:21:14 URL:https://raw.githubusercontent.com/NVIDIA/accelerated-computing-hub/refs/heads/main/tutorials/cuda-cpp/notebooks/01.03-Extending-Algorithms/Sources/ach.h [2787/2787] -> "Sources/ach.h" [1]


In [2]:
%%writefile Sources/naive-max-diff.cpp
#include "ach.h"

float naive_max_change(const thrust::universal_vector<float>& a,
                       const thrust::universal_vector<float>& b)
{
    // allocate vector to store `a - b`
    thrust::universal_vector<float> unnecessarily_materialized_diff(a.size());

    // compute products
    thrust::transform(thrust::device,
                      a.begin(), a.end(),                       // first input sequence
                      b.begin(),                                // second input sequence
                      unnecessarily_materialized_diff.begin(),  // result
                      []__host__ __device__(float x, float y) { // transformation (abs diff)
                         return abs(x - y);
                      });

    // compute max difference
    return thrust::reduce(thrust::device,
                          unnecessarily_materialized_diff.begin(),
                          unnecessarily_materialized_diff.end(),
                          0.0f, thrust::maximum<float>{});
}

int main()
{
    float k = 0.5;
    float ambient_temp = 20;
    thrust::universal_vector<float> temp[] = {{ 42, 24, 50 }, { 0, 0, 0}};
    auto transformation = [=] __host__ __device__ (float temp) { return temp + k * (ambient_temp - temp); };

    std::printf("step  max-change\n");
    for (int step = 0; step < 3; step++) {
        thrust::universal_vector<float> &current = temp[step % 2];
        thrust::universal_vector<float> &next = temp[(step + 1) % 2];

        thrust::transform(thrust::device, current.begin(), current.end(), next.begin(), transformation);
        std::printf("%d     %.2f\n", step, naive_max_change(current, next));
    }
}

Writing Sources/naive-max-diff.cpp


In [3]:
!nvcc --extended-lambda -o /tmp/a.out Sources/naive-max-diff.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

step  max-change
0     15.00
1     7.50
2     3.75


In [4]:
%%writefile Sources/pointer.cpp
#include "ach.h"

int main()
{
    std::array<int, 3> a{ 0, 1, 2 };

    int* pointer = a.data();

    std::printf("pointer[0]: %d\n", pointer[0]); // prints 0
    std::printf("pointer[1]: %d\n", pointer[1]); // prints 1
}

Writing Sources/pointer.cpp


In [5]:
!nvcc --extended-lambda -o /tmp/a.out Sources/pointer.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

pointer[0]: 0
pointer[1]: 1


In [6]:
%%writefile Sources/counting.cpp
#include "ach.h"

struct counting_iterator
{
  int operator[](int i)
  {
    return i;
  }
};

int main()
{
  counting_iterator it;

  std::printf("it[0]: %d\n", it[0]); // prints 0
  std::printf("it[1]: %d\n", it[1]); // prints 1
}

Writing Sources/counting.cpp


In [7]:
!nvcc --extended-lambda -o /tmp/a.out Sources/counting.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

it[0]: 0
it[1]: 1


In [8]:
%%writefile Sources/transform.cpp
#include "ach.h"

struct transform_iterator
{
  int *a;

  int operator[](int i)
  {
    return a[i] * 2;
  }
};

int main()
{
  std::array<int, 3> a{ 0, 1, 2 };

  transform_iterator it{a.data()};

  std::printf("it[0]: %d\n", it[0]); // prints 0 (0 * 2)
  std::printf("it[1]: %d\n", it[1]); // prints 2 (1 * 2)
}

Writing Sources/transform.cpp


In [9]:
!nvcc --extended-lambda -o /tmp/a.out Sources/transform.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

it[0]: 0
it[1]: 2


In [19]:
%%writefile Sources/zip.cpp
#include "ach.h"

struct zip_iterator
{
  int *a;
  int *b;

  std::tuple<int, int> operator[](int i)
  {
    return {a[i], b[i]};
  }
};

int main()
{
  std::array<int, 3> a{ 0, 1, 2 };
  std::array<int, 3> b{ 5, 4, 2 };

  zip_iterator it{a.data(), b.data()};

  std::printf("it[0]: (%d, %d)\n", std::get<0>(it[0]), std::get<1>(it[0])); // prints (0, 5)
  std::printf("it[0]: (%d, %d)\n", std::get<0>(it[1]), std::get<1>(it[1])); // prints (1, 4)
}

Overwriting Sources/zip.cpp


In [20]:
!nvcc --extended-lambda -o /tmp/a.out Sources/zip.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

it[0]: (0, 5)
it[0]: (1, 4)


In [ ]:
%%writefile Sources/transform-zip.cpp
#include "ach.h"

struct zip_iterator
{
  int *a;
  int *b;

  std::tuple<int, int> operator[](int i)
  {
    return {a[i], b[i]};
  }
};

struct transform_iterator
{
  zip_iterator zip;

  int operator[](int i)
  {
    auto [a, b] = zip[i];
    return abs(a - b);
  }
};

int main()
{
  std::array<int, 3> a{ 0, 1, 2 };
  std::array<int, 3> b{ 5, 4, 2 };

  zip_iterator zip{a.data(), b.data()};
  transform_iterator it{zip};

  std::printf("it[0]: %d\n", it[0]); // prints 5
  std::printf("it[0]: %d\n", it[1]); // prints 3
}

In [ ]:
!nvcc --extended-lambda -o /tmp/a.out Sources/transform-zip.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

In [ ]:
%%writefile Sources/transform-output.cpp
#include "ach.h"

struct wrapper
{
   int *ptr;

   void operator=(int value) {
      *ptr = value / 2;
   }
};

struct transform_output_iterator
{
  int *a;

  wrapper operator[](int i)
  {
    return {a + i};
  }
};

int main()
{
  std::array<int, 3> a{ 0, 1, 2 };
  transform_output_iterator it{a.data()};

  it[0] = 10;
  it[1] = 20;

  std::printf("a[0]: %d\n", a[0]); // prints 5
  std::printf("a[1]: %d\n", a[1]); // prints 10
}

In [ ]:
!nvcc --extended-lambda -o /tmp/a.out Sources/transform-output.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

In [ ]:
%%writefile Sources/discard.cpp
#include "ach.h"

struct wrapper
{
   void operator=(int value) {
      // discard value
   }
};

struct discard_iterator
{
  wrapper operator[](int i)
  {
    return {};
  }
};

int main()
{
  discard_iterator it{};

  it[0] = 10;
  it[1] = 20;
}

In [ ]:
!nvcc --extended-lambda -o /tmp/a.out Sources/discard.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

In [ ]:
%%writefile Sources/zip.cpp
#include "ach.h"

int main()
{
    // allocate and initialize input vectors
    thrust::universal_vector<float> a{ 31, 22, 35 };
    thrust::universal_vector<float> b{ 25, 21, 27 };

    // zip two vectors into a single iterator
    auto zip = thrust::make_zip_iterator(a.begin(), b.begin());

    thrust::tuple<float, float> first = *zip;
    std::printf("first: (%g, %g)\n", thrust::get<0>(first), thrust::get<1>(first));

    zip++;

    thrust::tuple<float, float> second = *zip;
    std::printf("second: (%g, %g)\n", thrust::get<0>(second), thrust::get<1>(second));
}

In [ ]:
!nvcc --extended-lambda -o /tmp/a.out Sources/zip.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

In [ ]:
%%writefile Sources/transform.cpp
#include "ach.h"

int main()
{
    thrust::universal_vector<float> a{ 31, 22, 35 };
    thrust::universal_vector<float> b{ 25, 21, 27 };

    auto zip = thrust::make_zip_iterator(a.begin(), b.begin());
    auto transform = thrust::make_transform_iterator(zip, []__host__ __device__(thrust::tuple<float, float> t) {
        return abs(thrust::get<0>(t) - thrust::get<1>(t));
    });

    std::printf("first: %g\n", *transform); // absolute difference of `a[0] = 31` and `b[0] = 25`

    transform++;

    std::printf("second: %g\n", *transform); // absolute difference of `a[1] = 22` and `b[1] = 21`
}

In [ ]:
!nvcc --extended-lambda -o /tmp/a.out Sources/transform.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

In [ ]:
%%writefile Sources/optimized-max-diff.cpp
#include "ach.h"

float max_change(const thrust::universal_vector<float>& a,
                 const thrust::universal_vector<float>& b)
{
    auto zip = thrust::make_zip_iterator(a.begin(), b.begin());
    auto transform = thrust::make_transform_iterator(zip, []__host__ __device__(thrust::tuple<float, float> t) {
        return abs(thrust::get<0>(t) - thrust::get<1>(t));
    });

    // compute max difference
    return thrust::reduce(thrust::device, transform, transform + a.size(), 0.0f, thrust::maximum<float>{});
}

int main()
{
    float k = 0.5;
    float ambient_temp = 20;
    thrust::universal_vector<float> temp[] = {{ 42, 24, 50 }, { 0, 0, 0}};
    auto transformation = [=] __host__ __device__ (float temp) { return temp + k * (ambient_temp - temp); };

    std::printf("step  max-change\n");
    for (int step = 0; step < 3; step++) {
        thrust::universal_vector<float> &current = temp[step % 2];
        thrust::universal_vector<float> &next = temp[(step + 1) % 2];

        thrust::transform(thrust::device, current.begin(), current.end(), next.begin(), transformation);
        std::printf("%d     %.2f\n", step, max_change(current, next));
    }
}

In [ ]:
!nvcc --extended-lambda -o /tmp/a.out Sources/optimized-max-diff.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

In [2]:
%%writefile Sources/naive-vs-iterators.cpp
#include "ach.h"

float naive_max_change(const thrust::universal_vector<float>& a,
                       const thrust::universal_vector<float>& b)
{
    thrust::universal_vector<float> diff(a.size());
    thrust::transform(thrust::device, a.begin(), a.end(), b.begin(), diff.begin(),
                      []__host__ __device__(float x, float y) {
                         return abs(x - y);
                      });
    return thrust::reduce(thrust::device, diff.begin(), diff.end(), 0.0f, thrust::maximum<float>{});
}

float max_change(const thrust::universal_vector<float>& a,
                 const thrust::universal_vector<float>& b)
{
    auto zip = thrust::make_zip_iterator(a.begin(), b.begin());
    auto transform = thrust::make_transform_iterator(zip, []__host__ __device__(thrust::tuple<float, float> t) {
        return abs(thrust::get<0>(t) - thrust::get<1>(t));
    });
    return thrust::reduce(thrust::device, transform, transform + a.size(), 0.0f, thrust::maximum<float>{});
}

int main()
{
    // allocate vectors containing 2^28 elements
    thrust::universal_vector<float> a(1 << 28);
    thrust::universal_vector<float> b(1 << 28);

    thrust::sequence(a.begin(), a.end());
    thrust::sequence(b.rbegin(), b.rend());

    auto start_naive = std::chrono::high_resolution_clock::now();
    naive_max_change(a, b);
    auto end_naive = std::chrono::high_resolution_clock::now();
    const double naive_duration = std::chrono::duration_cast<std::chrono::milliseconds>(end_naive - start_naive).count();

    auto start = std::chrono::high_resolution_clock::now();
    max_change(a, b);
    auto end = std::chrono::high_resolution_clock::now();
    const double duration = std::chrono::duration_cast<std::chrono::milliseconds>(end - start).count();

    std::printf("iterators are %g times faster than naive approach\n", naive_duration / duration);
}

Writing Sources/naive-vs-iterators.cpp


In [3]:
!nvcc --extended-lambda -o /tmp/a.out Sources/naive-vs-iterators.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

iterators are 23.2222 times faster than naive approach


In [9]:
%%writefile Sources/variance.cpp
#include "ach.h"

float variance(const thrust::universal_vector<float> &x, float mean) {
  // TODO: Update the following line so that dereferencing
  // `squared_differences` returns `(xi - mean) * (xi - mean)`
  auto squared_differences = thrust::make_transform_iterator(
  x.begin(), [mean] __host__ __device__(float value) {
    return (value - mean) * (value - mean);
  });

  return thrust::reduce(thrust::device, squared_differences,
                        squared_differences + x.size()) /
         x.size();
}

float mean(thrust::universal_vector<float> vec) {
  return thrust::reduce(thrust::device, vec.begin(), vec.end()) / vec.size();
}

int main() {
  float ambient_temp = 20;
  thrust::universal_vector<float> prev{42, 24, 50};
  thrust::universal_vector<float> next{0, 0, 0};

  std::printf("step  variance\n");
  for (int step = 0; step < 3; step++) {
    thrust::transform(thrust::device, prev.begin(), prev.end(), next.begin(),
                      [=] __host__ __device__(float temp) {
                        return temp + 0.5 * (ambient_temp - temp);
                      });
    std::printf("%d     %.2f\n", step, variance(next, mean(next)));
    next.swap(prev);
  }
}

Overwriting Sources/variance.cpp


In [10]:
!nvcc --extended-lambda -o /tmp/a.out Sources/variance.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

step  variance
0     29.56
1     7.39
2     1.85
